# SimplesConta
Seja bem vindo ao SimplesConta!

Aqui buscamos, por meio de uma ferramenta simples, te ajudar a registrar receitas e despesas, acompanhar saldo mensal e gerar relatórios financeiros.

In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
from datetime import datetime
import csv
import os
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
# Lista para armazenar as transações
transacoes = []

In [3]:
# Categorias pré-definidas
categorias_padrao = ["Salário", "Alimentação", "Transporte", "Lazer", "Educação", "Outros"]

In [4]:
# Caminho do arquivo CSV
ARQUIVO_DADOS = "transacoes.csv"

In [5]:
# Função para carregar dados do CSV
def carregar_dados():
    if os.path.exists(ARQUIVO_DADOS):
        with open(ARQUIVO_DADOS, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                row['valor'] = float(row['valor'])
                transacoes.append(row)
        atualizar_lista()
        atualizar_saldo()

In [6]:
# Função para salvar dados no CSV
def salvar_dados():
    with open(ARQUIVO_DADOS, mode='w', newline='') as file:
        campos = ["tipo", "valor", "categoria", "descricao", "data"]
        writer = csv.DictWriter(file, fieldnames=campos)
        writer.writeheader()
        for t in transacoes:
            writer.writerow(t)

In [7]:
# Função para adicionar transação
def adicionar_transacao():
    tipo = tipo_var.get()
    valor = valor_entry.get()
    categoria = categoria_combo.get()
    descricao = descricao_entry.get()

    try:
        valor = float(valor)
        data = datetime.now().strftime("%d/%m/%Y")
        transacao = {
            "tipo": tipo,
            "valor": valor,
            "categoria": categoria,
            "descricao": descricao,
            "data": data
        }
        transacoes.append(transacao)
        salvar_dados()
        atualizar_lista()
        atualizar_saldo()
        limpar_campos()
    except ValueError:
        messagebox.showerror("Erro", "Digite um valor válido.")

In [8]:
# Atualiza a lista de transações
def atualizar_lista(filtro_data=None, filtro_tipo=None):
    lista_transacoes.delete(0, tk.END)
    for t in transacoes:
        if (filtro_data and filtro_data not in t['data']) or (filtro_tipo and filtro_tipo != t['tipo']):
            continue
        lista_transacoes.insert(tk.END, f"{t['data']} - {t['tipo']} - R${t['valor']:.2f} - {t['categoria']} - {t['descricao']}")

In [9]:
# Calcula e atualiza o saldo
def atualizar_saldo():
    saldo = 0
    for t in transacoes:
        if t['tipo'] == 'Receita':
            saldo += t['valor']
        else:
            saldo -= t['valor']
    saldo_label.config(text=f"Saldo Atual: R${saldo:.2f}")

In [10]:
# Limpa os campos do formulário
def limpar_campos():
    valor_entry.delete(0, tk.END)
    categoria_combo.set('')
    descricao_entry.delete(0, tk.END)

In [11]:
# Função para aplicar filtros
def aplicar_filtros():
    data = filtro_data_entry.get()
    tipo = filtro_tipo_combo.get()
    if tipo == 'Todos':
        tipo = None
    atualizar_lista(filtro_data=data if data else None, filtro_tipo=tipo)

In [12]:
# Função para exibir gráfico de despesas por categoria
def mostrar_grafico():
    categorias = defaultdict(float)
    for t in transacoes:
        if t['tipo'] == 'Despesa':
            categorias[t['categoria']] += t['valor']

    if categorias:
        labels = list(categorias.keys())
        valores = list(categorias.values())

        plt.figure(figsize=(6, 6))
        plt.pie(valores, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.title('Distribuição de Despesas por Categoria')
        plt.axis('equal')
        plt.show()
    else:
        messagebox.showinfo("Info", "Nenhuma despesa cadastrada para exibir o gráfico.")

In [13]:
app = tk.Tk()
app.title("Gestão Financeira Pessoal")

''

In [14]:
# Adicionando cabeçalho
header_label = tk.Label(app, text="Seja bem-vindo ao SimplesConta!", font=("Arial", 16, "bold"))
header_label.grid(row=0, column=0, columnspan=3, pady=10)

In [15]:
# Adicionando texto inicial
intro_label = tk.Label(app, text="Aqui buscamos, por meio de uma ferramenta simples, te ajudar a registrar receitas e despesas, acompanhar saldo mensal e gerar relatórios financeiros.",
                       wraplength=400, justify="left", font=("Arial", 12))
intro_label.grid(row=1, column=0, columnspan=3, pady=5)

In [16]:
# Tipo de transação
tipo_var = tk.StringVar(value="Despesa")
tk.Label(app, text="Tipo:").grid(row=13, column=0)
tk.Radiobutton(app, text="Receita", variable=tipo_var, value="Receita").grid(row=12, column=1)
tk.Radiobutton(app, text="Despesa", variable=tipo_var, value="Despesa").grid(row=12, column=2)

In [17]:
# Campo valor
tk.Label(app, text="Valor:").grid(row=13, column=0)
valor_entry = tk.Entry(app)
valor_entry.grid(row=13, column=1, columnspan=2)

In [18]:
# Campo categoria
tk.Label(app, text="Categoria:").grid(row=14, column=0)
categoria_combo = ttk.Combobox(app, values=categorias_padrao)
categoria_combo.grid(row=14, column=1, columnspan=2)

In [19]:
# Campo descrição
tk.Label(app, text="Descrição:").grid(row=15, column=0)
descricao_entry = tk.Entry(app)
descricao_entry.grid(row=15, column=1, columnspan=2)

In [20]:
# Botão adicionar
tk.Button(app, text="Adicionar Transação", command=adicionar_transacao).grid(row=18, column=0, columnspan=3)

In [21]:
# Filtros
tk.Label(app, text="Filtrar por data (dd/mm/aaaa):").grid(row=20, column=0)
filtro_data_entry = tk.Entry(app)
filtro_data_entry.grid(row=20, column=1)

filtro_tipo_combo = ttk.Combobox(app, values=["Todos", "Receita", "Despesa"])
filtro_tipo_combo.set("Todos")
filtro_tipo_combo.grid(row=21, column=2)
tk.Button(app, text="Aplicar Filtros", command=aplicar_filtros).grid(row=21, column=0, columnspan=3)

In [22]:
# Lista de transações
lista_transacoes = tk.Listbox(app, width=80)
lista_transacoes.grid(row=23, column=0, columnspan=3)

In [23]:
# Saldo
saldo_label = tk.Label(app, text="Saldo Atual: R$0.00")
saldo_label.grid(row=24, column=0, columnspan=3)

In [24]:
# Botão gráfico
tk.Button(app, text="Mostrar Gráfico de Despesas", command=mostrar_grafico).grid(row=9, column=0, columnspan=3, pady=10)

In [25]:
# Iniciar aplicação
carregar_dados()
app.mainloop()